In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 19
TITLE = "8 February 2021"
SUBTITLE = "8 AM 9 February 2021"
sts = utc(2021, 2, 8, 17)
ets = utc(2021, 2, 9, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [9]:
cull = ['IA-DL-33', 'RMSI4']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=3, upper=4)
res = mp.postprocess(filename='210209.png')
mp.close()

    state  wfo  val       lon      lat                               geo  \
215    AK  AJK  3.8 -131.6900  55.3600  POINT (-1939725.867 1557476.477)   
216    IN  IND  3.8  -86.2300  39.2800   POINT (1179955.192 -537545.762)   
217    IA  DVN  3.8  -90.5400  41.6900    POINT (783685.035 -322815.383)   
218    IL  LOT  3.8  -87.7800  41.7400   POINT (1009664.318 -287111.152)   
219    OH  ILN  3.8  -84.4200  39.1400   POINT (1335675.400 -525425.295)   
..    ...  ...  ...       ...      ...                               ...   
533    IN  ILN  3.0  -84.9302  39.6323   POINT (1283142.638 -479448.913)   
534    IL  LSX  3.0  -88.9461  38.6333    POINT (958144.879 -643847.162)   
535    IL  PAH  3.0  -88.3117  38.6058   POINT (1013101.443 -639360.120)   
536    IL  LOT  3.0  -88.3500  41.7500    POINT (962819.899 -292866.473)   
537    PA  PBZ  3.0  -79.6681  40.6842   POINT (1694063.924 -269046.781)   

     used_for_analysis  nwsli  plotme source  
215               True    215    True   